In [7]:
import time
import numpy as np
import torch
from d2l import torch as d2l
from torch import nn

Module 类里定义 模型 损失函数 和 优化器
DataModule 类里定义 数据加载器 dataLoader
Trainer 类组合，在硬件上训练模型

In [8]:
#往类里加功能，甚至是已经创建的实例
#用到了wrapper函数，这个函数返回的是一个函数，这个函数的参数是一个函数，这个函数的功能是把这个函数加到类里，装饰器的功能
def add_to_class(Class):
    '''Register functions as methods in created class'''
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)#obj是函数
    return wrapper

In [9]:
class A:
    def __init__(self) -> None:
        self.b = 1
a = A()


In [10]:
@add_to_class(A)
def do(self):
    print('class attrribute "b" is ',self.b)

a.do()

class attrribute "b" is  1


In [21]:
class HyperParameters:
    def save_hyperparameters(self, ignore=[]):
        raise NotImplemented
 


self.a=1, self.b=2, self.c=3
There is no self.c True


In [25]:
class B(d2l.HyperParameters):
    def __init__(self, a, b, c) -> None:
        super().__init__()
        self.save_hyperparameters(ignore=['c'])
        print(f'self.a={a}, self.b={b}, self.c={c}', )
        print(f'There is no self.c', not hasattr(self, 'c'))

b = B(a=1,b=2,c=5)    

self.a=1, self.b=2, self.c=5
There is no self.c True


In [34]:
class ProgressBoard(d2l.HyperParameters):
    def __init__(self,  xlabel=None, ylabel=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 ls=['-', '--', '-.', ':'], colors=['C0', 'C1', 'C2', 'C3'],
                 fig=None, axes=None, figsize=(3.5, 2.5), display=True) -> None:
        super().__init__()
        self.save_hyperparameters()
    def draw(self, x, y, label, every_n=1):
        raise NotImplemented    

In [ ]:
board = d2l.ProgressBoard('x')
for x in np.arange(0, 10, 0.1):
    board.draw(x, np.sin(x), 'sin', every_n=10)
    board.draw(x, np.cos(x), 'cos', every_n=1)

In [35]:
class Module(nn.Module, d2l.HyperParameters):
    def __init__(self, plot_train_epoch=2, plot_valid_per_epoch=1) -> None:
        super().__init__()
        self.save_hyperparameters()
        self.board = ProgressBoard()
        
    def loss(self, y_hat, y):
        raise NotImplemented
    
    def forward(self, X):
        assert hasattr(self, 'net'),'Neural network is not defined'
        return self.net(X)
    
    def plot(self, key, value, train):
        assert hasattr(self, 'trainer'), 'Trainer is not inited'
        self.board.xlabel = 'epoch'
        if train:
            
            x = self.trainer.train_epoch_idx / \
                self.trainer.num_train_batches
            n = self.trainer.num_train_batches /\
                self.plot_train_epoch#n是每个epoch画几次
        
        else:
            x = self.trainer.epoch + 1
            n = self.trainer.num_val_batches / \
                self.plot_valid_per_epoch
        
        self.board.draw(x, value.to(d2l.cpu()).detach().numpy(),
                        ('train_' if train else 'val_') + key,
                        every_n=int(n))
    def training_step(self, batch):
        l = self.loss(self(*batch[:-1]), batch[-1])
        self.plot('loss', l, train=True)
        return l

    def validation_step(self, batch):
        l = self.loss(self(*batch[:-1]), batch[-1])
        self.plot('loss', l, train=False)
    
    def configure_optimizers(self):
        raise NotImplementedError

In [ ]:
class DataModule(d2l.HyperParameters):
    def __init__(self, root='..\data', num_workers=4):
        self.save_hyperparameters()
        
    def get_dataloader(self, train):
        raise NotImplementedError
    
    def train_dataloader(self):
        return self.get_dataloader(train=True)
    
    def val_dataloader(self):
        return self.get_dataloader(train=False)
    

class Trainer(d2l.HyperParameters):
    def __init__(self, max_epochs, num_gpus) -> None:
        super().__init__()   
        self.save_hyperparameters()
        assert num_gpus == 0 or torch.cuda.is_available(), 'No GPU is available'
    
    def prepare_data(self, data):
        self.train_loader = data.train_dataloader()
        self.val_loader = data.val_dataloader()
        self.num_train_batches = len(self.train_loader)
        self.num_val_batches = len(self.val_loader) if self.val_loader else 0
        
    def prepare_model(self, model):
        self.trainer = self
        model.board.xlim = [0, self.max_epochs]
        self.model = model


    def fit(self, model,data):
        self.prepare_data(data)
        self.prepare_model(model)
        self.optim = model.configure_optimizers()
        self.epoch = 0
        self.train_batch_idx = 0
        self.val_batch_idx = 0
        for self.epoch in range(self.max_epochs):
            self.fit_epoch()
    
    def fit_epoch(self):
        raise NotImplementedError            